### Demo notebook for transmission analysis

* Calculate which branches or transmission lines are congested by counting the fraction of total hours that the power flow is 0.75, 0.8 or 0.9 of the branch capacity. Z-score and p-values are calculated for the 0.75 case, since 0.75 is also the value used by WECC (useful for validation).


In [1]:
import pandas as pd
import numpy as np

import os
import sys
sys.path.append("..")
import westernintnet

import transmission

In [2]:
wecc = westernintnet.WesternIntNet()

Loading sub
Loading bus2sub
Loading bus
Loading genbus
Loading branches
Loading resources
Loading net_generation
Load solar data
Load wind data
Load hydro data
Load demand data
Done loading


In [3]:
dir1 = os.path.join('..','test','data')

* Power flow outputs are usually saved as pkl files. These are large files and are not saved in github.

In [5]:
PF = pd.read_pickle(dir1 + '/' + 'power flow pkl file')

In [6]:
wecc.PF_base = PF
wecc.branches.index = PF.columns

#Create Capacity column; Note that rateA == 0 represents low impedance lines
#Denoting rateA==0 is simply a signal to MatPower that there are no current constraints on the line.
wecc.branches.loc[wecc.branches.rateA != 0, 'Capacity'] = wecc.branches['rateA']
wecc.branches.loc[wecc.branches.rateA == 0, 'Capacity'] = 99999.

#Normalize power flow values to line capacity
wecc.congestion_base = wecc.PF_base.div(wecc.branches['Capacity']).apply(np.abs)  

#Create congestion statistics file
wecc.congestion_stats_base = transmission.generate_cong_stats(wecc.congestion_base, wecc.branches, dir1 + '/' + 'congestion_stats_base_demo')

In [17]:
wecc.congestion_stats_base.loc[wecc.congestion_stats_base['pvalue']<0.05].sort_values(['pvalue', 'zscore'], ascending=False).head(10)

,Capacity,hutil1,hutil0p9-1,hutil0p8-0p9,hutil0p75-0p8,hutil>=0p9,hutil>=0p8,hutil>=0p75,dist,zscore,pvalue
line,,,,,,,,,,,
2952,11,0,0,0,28,0,0,28,0.000,1.652440,0.049222
9583,246,0,0,0,30,0,0,30,27.471,2.094437,0.018111
12376,170,0,0,10,20,0,10,30,45.728,2.094437,0.018111
6541,208,0,0,1,30,0,1,31,4.209,2.315435,0.010295
7385,167,0,0,0,33,0,0,33,0.000,2.757432,0.002913
10338,151,0,2,5,26,2,7,33,0.000,2.757432,0.002913
1547,187,0,1,14,20,1,15,35,31.502,3.199429,0.000689
11469,51,0,0,0,35,0,0,35,0.000,3.199429,0.000689
6364,185,0,0,0,36,0,0,36,18.838,3.420427,0.000313
